In [1]:
# Read data_clean
import pandas as pd

df = pd.read_json('./data_clean/data_clean.json', orient="records")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7565 entries, 0 to 7564
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     7565 non-null   int64  
 1   keywords               7565 non-null   object 
 2   characters             7565 non-null   object 
 3   actors                 7565 non-null   object 
 4   director               7565 non-null   object 
 5   crew                   7565 non-null   object 
 6   adult                  7565 non-null   object 
 7   belongs_to_collection  7565 non-null   object 
 8   budget                 7565 non-null   int64  
 9   genres                 7565 non-null   object 
 10  homepage               1303 non-null   object 
 11  imdb_id                7563 non-null   object 
 12  original_language      7565 non-null   object 
 13  original_title         7565 non-null   object 
 14  overview               7565 non-null   object 
 15  popu

In [2]:
def preprocess_row(row):
    # Unir las características adicionales en una sola cadena de texto
    genre = row["genres"]
    keywords = row["keywords"]
    overview = row["overview"]
    director = row["director"]
    actor = row["actors"]
    combined_text = f" {genre} [SEP] keywords: {keywords} [SEP] overview: {overview} [SEP] director: {director} [SEP] actors: {actor}"
    return combined_text

In [3]:
import torch
from transformers import BertModel, BertTokenizer
import pandas as pd
import faiss
import numpy as np
from tqdm import tqdm  # Importar tqdm para la barra de progreso
import time  # Importar time para calcular el tiempo transcurrido

# Cargar modelo y tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Función para obtener embeddings por lote
def get_bert_embeddings(texts, batch_size=32):
    embeddings = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Mover el modelo a GPU si está disponible

    # Inicializar tqdm para la barra de progreso
    pbar = tqdm(total=len(texts), desc="Processing texts", unit="texts")
    start_time = time.time()  # Tiempo de inicio del procesamiento

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i : i + batch_size]
        tokens = tokenizer(
            batch_texts, padding=True, truncation=True, return_tensors="pt"
        ).to(device)
        with torch.no_grad():
            outputs = model(**tokens)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1)
        embeddings.append(batch_embeddings)

        # Actualizar la barra de progreso
        pbar.update(len(batch_texts))

        # Calcular tiempo transcurrido y estimar tiempo restante
        elapsed_time = time.time() - start_time
        texts_per_sec = (i + len(batch_texts)) / elapsed_time
        remaining_time = (len(texts) - (i + len(batch_texts))) / texts_per_sec

        # Actualizar descripción de la barra de progreso con el tiempo estimado
        pbar.set_postfix({"ETA": f"{remaining_time:.1f} sec"})

    pbar.close()  # Cerrar la barra de progreso al finalizar
    return torch.cat(embeddings, dim=0).cpu().numpy()

/home/felipe/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df['combined'] = df.apply(preprocess_row, axis=1)
combined_texts = df['combined'].tolist()

In [5]:
# Obtener embeddings
embeddings_np = get_bert_embeddings(combined_texts, batch_size=32)

Processing texts: 100%|██████████| 7565/7565 [11:16<00:00, 11.18texts/s, ETA=0.0 sec]  


In [15]:
# Crear el índice en FAISS
d = embeddings_np.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings_np)

# Función para obtener recomendaciones con FAISS
import numpy as np

def get_recommendations_faiss(titles, df, index, embeddings, k=10):
    # Obtener los índices de las películas que coinciden con los títulos
    indices = [df[df['title'] == title].index[0] for title in titles]
    
    # Obtener los embeddings promedio de las películas
    avg_embedding = np.mean(embeddings[indices], axis=0, keepdims=True)
    # Buscar los k vecinos más cercanos al embedding promedio
    D, I = index.search(avg_embedding, k)
    
    # I[0][1:] contiene los índices de las películas más similares (excluyendo las ingresadas)
    movie_indices = I[0][1:]
    print(movie_indices)
    # Devolver los títulos de las películas recomendadas
    return df['title'].iloc[movie_indices].tolist()


# Ejemplo de uso
recommended_movies_faiss = get_recommendations_faiss(['Ratatouille',"Toy Story"], df, index, embeddings_np, k=15)
print(recommended_movies_faiss)


[3049  608  549 1631  579  660   96 1231    0 2857  976 2787  838 1101]
['Bee Movie', 'An American Tail', 'Doctor Dolittle', 'Finding Nemo', 'Gremlins 2: The New Batch', 'Pleasantville', 'The Flintstones', 'The Great Outdoors', 'Toy Story', 'Ratatouille', 'Return to Me', 'Open Season', 'Home Alone 2: Lost in New York', 'How the Grinch Stole Christmas']


In [20]:
df.iloc[[3049, 608]]["id"]

3049    5559
608     4978
Name: id, dtype: int64

In [9]:
# Guardar embeddings y el índice en FAISS
np.save('./assets/embeddings.npy', embeddings_np)

d = embeddings_np.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings_np)
faiss.write_index(index, './assets/embedding_index.faiss')